In [ ]:
#This file is a new attempt to create a cleaned ingredients from extracted veg recipes (100 recipes as a sample dataset)

#--This file is used to create a sample 100recipes for training with a new column for just the ingredients extracted from the ingredients column
#The last part of code has a new approach on NER that also contains ignoring common ingredients that everyone has at home

Step 4:   In this file we exolored a new approach to extract just the ingredients from the given list of ingredients to prepare the data for training the model. This is ran on the dataset of 100 samples from recipes_reduced_veg_recipes.csv

In [ ]:
import numpy as np
import itertools
import pandas as pd
import json
import sys
import seaborn as sb
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import time

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# #Read csv for ingredients column

# # df = pd.read_csv('/content/drive/MyDrive/recipes_reduced_veg_recipes.csv')
# df.head()

,title,ingredients,instructions
0,dilly macaroni salad recipe,"['1 c. elbow macaroni', '1 c. cubed American c...",['Cook macaroni according to package direction...
1,gazpacho,"['8 tomatoes, quartered', 'Kosher salt', '1 re...",['Add the tomatoes to a food processor with a ...
2,crunchy onion potato bake,"['2 12 cups milk', '1 12 cups water', '14 cup ...","['Preheat oven to 350 degrees Fahrenheit.', 'S..."
3,cool 'n easy creamy watermelon pie,"['1 (3 ounce) package watermelon gelatin', '14...","['Dissolve Jello in boiling water.', 'Allow to..."
4,strawberry rhubarb dump cake,"['6 -8 cups fresh rhubarb, or', '6 -8 cups fro...",['Put ingredients in a buttered 9 x 12 x 2-inc...


# New Section

In [ ]:
#Read csv 
df = pd.read_csv('/content/drive/MyDrive/Project 298/Final Project Files/EDA Notebooks/recipes_reduced_veg_recipes.csv')
df.head()

,title,ingredients,instructions
0,dilly macaroni salad recipe,"['1 c. elbow macaroni', '1 c. cubed American c...",['Cook macaroni according to package direction...
1,gazpacho,"['8 tomatoes, quartered', 'Kosher salt', '1 re...",['Add the tomatoes to a food processor with a ...
2,crunchy onion potato bake,"['2 12 cups milk', '1 12 cups water', '14 cup ...","['Preheat oven to 350 degrees Fahrenheit.', 'S..."
3,cool 'n easy creamy watermelon pie,"['1 (3 ounce) package watermelon gelatin', '14...","['Dissolve Jello in boiling water.', 'Allow to..."
4,strawberry rhubarb dump cake,"['6 -8 cups fresh rhubarb, or', '6 -8 cups fro...",['Put ingredients in a buttered 9 x 12 x 2-inc...


In [ ]:
df_sample_2= df.head(5000)

In [ ]:
df_sample_2.head()

,title,ingredients,instructions
0,dilly macaroni salad recipe,"['1 c. elbow macaroni', '1 c. cubed American c...",['Cook macaroni according to package direction...
1,gazpacho,"['8 tomatoes, quartered', 'Kosher salt', '1 re...",['Add the tomatoes to a food processor with a ...
2,crunchy onion potato bake,"['2 12 cups milk', '1 12 cups water', '14 cup ...","['Preheat oven to 350 degrees Fahrenheit.', 'S..."
3,cool 'n easy creamy watermelon pie,"['1 (3 ounce) package watermelon gelatin', '14...","['Dissolve Jello in boiling water.', 'Allow to..."
4,strawberry rhubarb dump cake,"['6 -8 cups fresh rhubarb, or', '6 -8 cups fro...",['Put ingredients in a buttered 9 x 12 x 2-inc...


Perform Lemmatization to convert ingredients to their root form


In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def lemmatize(row):
    doc = nlp(row['ingredients'])
    lemmatized_ingredients = " ".join(token.lemma_ for token in doc)
    #print('-------------------------')
    #print(row['ingredients'])
    #print(lemmatized_ingredients)
    return lemmatized_ingredients

In [ ]:
df_sample_2['ingredients_lemma'] = df_sample_2.apply(lambda x: lemmatize(x), axis=1)

<ipython-input-8-334421f119ee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_2['ingredients_lemma'] = df_sample_2.apply(lambda x: lemmatize(x), axis=1)


In [ ]:
df_sample_2.head()

,title,ingredients,instructions,ingredients_lemma
0,dilly macaroni salad recipe,"['1 c. elbow macaroni', '1 c. cubed American c...",['Cook macaroni according to package direction...,"[ ' 1 c. elbow macaroni ' , ' 1 c. cube americ..."
1,gazpacho,"['8 tomatoes, quartered', 'Kosher salt', '1 re...",['Add the tomatoes to a food processor with a ...,"[ ' 8 tomato , quarter ' , ' kosher salt ' , '..."
2,crunchy onion potato bake,"['2 12 cups milk', '1 12 cups water', '14 cup ...","['Preheat oven to 350 degrees Fahrenheit.', 'S...","[ ' 2 12 cup milk ' , ' 1 12 cup water ' , ' 1..."
3,cool 'n easy creamy watermelon pie,"['1 (3 ounce) package watermelon gelatin', '14...","['Dissolve Jello in boiling water.', 'Allow to...",[ ' 1 ( 3 ounce ) package watermelon gelatin '...
4,strawberry rhubarb dump cake,"['6 -8 cups fresh rhubarb, or', '6 -8 cups fro...",['Put ingredients in a buttered 9 x 12 x 2-inc...,"[ ' 6 -8 cup fresh rhubarb , or ' , ' 6 -8 cup..."


In [ ]:
df_sample_2.drop(['ingredients'], axis=1, inplace=True)

<ipython-input-10-4bc109bbbff6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_2.drop(['ingredients'], axis=1, inplace=True)


In [ ]:
df_sample_2.head()

,title,instructions,ingredients_lemma
0,dilly macaroni salad recipe,['Cook macaroni according to package direction...,"[ ' 1 c. elbow macaroni ' , ' 1 c. cube americ..."
1,gazpacho,['Add the tomatoes to a food processor with a ...,"[ ' 8 tomato , quarter ' , ' kosher salt ' , '..."
2,crunchy onion potato bake,"['Preheat oven to 350 degrees Fahrenheit.', 'S...","[ ' 2 12 cup milk ' , ' 1 12 cup water ' , ' 1..."
3,cool 'n easy creamy watermelon pie,"['Dissolve Jello in boiling water.', 'Allow to...",[ ' 1 ( 3 ounce ) package watermelon gelatin '...
4,strawberry rhubarb dump cake,['Put ingredients in a buttered 9 x 12 x 2-inc...,"[ ' 6 -8 cup fresh rhubarb , or ' , ' 6 -8 cup..."


rename the column 


In [ ]:
df_sample_2.rename({'ingredients_lemma': 'ingredients'}, axis=1, inplace=True)

<ipython-input-12-6498a11042b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_2.rename({'ingredients_lemma': 'ingredients'}, axis=1, inplace=True)


In [ ]:
df_sample_2.head()

,title,instructions,ingredients
0,dilly macaroni salad recipe,['Cook macaroni according to package direction...,"[ ' 1 c. elbow macaroni ' , ' 1 c. cube americ..."
1,gazpacho,['Add the tomatoes to a food processor with a ...,"[ ' 8 tomato , quarter ' , ' kosher salt ' , '..."
2,crunchy onion potato bake,"['Preheat oven to 350 degrees Fahrenheit.', 'S...","[ ' 2 12 cup milk ' , ' 1 12 cup water ' , ' 1..."
3,cool 'n easy creamy watermelon pie,"['Dissolve Jello in boiling water.', 'Allow to...",[ ' 1 ( 3 ounce ) package watermelon gelatin '...
4,strawberry rhubarb dump cake,['Put ingredients in a buttered 9 x 12 x 2-inc...,"[ ' 6 -8 cup fresh rhubarb , or ' , ' 6 -8 cup..."


In [ ]:
# https://github.com/vishwapardeshi/NL_Parser_using_Spacy/blob/master/notebooks/Recipe_Ingredient_Parser.ipynb
# measurements = re.compile(r'(bowl|bulb|cube|clove|cup|drop|ounce|oz|pinch|pound|teaspoon|tablespoon|c.|small|large|medium|long|extralarge|extra|whole|inch|cut|mashed|chopped|minced|smashed|grated|quartered|boiling|can|canned| mix|layer|ground|shredded|trimmed|pieces|freshly|slivered|thinly|divided|halved|regular|cubes|peeled|uncooked|diced|unbaked|skinned|melted|finely|thawed|thickly|fresh|quartered|peeled|slices|dried|chilled|softened|warmed|bite|size|cubed|fluid|semihard|semisoft|c|dry|beaten|zest|sifted|julienned|slightly|beaten|thin|wedges|thick|strips|fine|tbsp|tsp|sliced|cup|cups|ounces|lb|lbs|pound|g|kg|grm)s?')


In [ ]:
# base_model = spacy.load('en_core_web_sm')
# ing_final['ingredients']= ing_final['ingredients'].astype(str)
# extracted = []

# for ix, row in ing_final.iterrows():
#     print('\r', "Extracting ingredient for row", ix, end='')
#     tokens = base_model(row['ingredients'])
#     extract = ''
#     for token in tokens:
#         if (token.dep_ in ['nsubj', 'ROOT']) and (token.pos_ in ['NOUN', 'PROPN']) and (not measurements.match(token.text)):
#         #explore children
#             for child in token.children:
#                 if (not measurements.match(child.text)) and (child.dep_ in ['amod', 'compound']):
#                     extract += child.text + ' '
#             extract += token.text + ' '
#     extracted.append(extract)

 Extracting ingredient for row 99

In [ ]:
# ing_final['cleaned ingredients'] = extracted

In [ ]:
# df_sample_2.to_csv('/content/drive/MyDrive/Project 298/Final Project Files/EDA Notebooks/sample_with_ingredients_extracted.csv', index=False)

Below code will help us solve problem

In [ ]:
import spacy, pprint, re
from spacy import displacy
from fractions import Fraction

In [ ]:
#https://github.com/wenzstev/grocery-list-maker/blob/master/recipe_parser.py

class RecipeNLP:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def parse(self, recipe_line):
        return ParsedLine(self.nlp, recipe_line)


class ParsedLine:
    def __init__(self, nlp, recipe_line):
        nlp_ing = nlp(recipe_line)
        self.measurement_regex = re.compile('(cup|tablespoon|teaspoon|oz|pound|ounce|clove|cube)s?')
# c.|tbsp|tsp|sliced|cup|cups|ounces|lb|lbs|pound|g|kg|grm|Tbsp|grams|gm|pint|tsp|c
        self.ingredient = ""
        self.measurement = ""
        self.amount = 0

        print()
        print(recipe_line)

        for token in nlp_ing:
            print(token, token.dep_, token.pos_)

            # TODO: determine the amount
            if self.measurement_regex.search(token.text):
                # print("the measurement is {}".format(token))
                self.measurement = self.measurement_regex.search(token.text).group(1)
            # TODO: determine the ingredient
            elif token.dep_ in ['ROOT', 'nsubj', 'dobj', 'appos']:
                if token.pos_ in ['NOUN', 'PROPN']:
                    # print("the ingredient is {}".format(token))

                    if token.lower_ in ['salt', 'water','ice','oil','']:   # ignore certain things that pretty much everyone has
                        # print('ingredient matches "ignore" list')
                        continue


                    # TODO: check if there are any compound words
                    # print(list(token.children))
                    # print('checking children for compounds')
                    for child in token.children:
                        # print('checking {}'.format(child))
                        if not self.measurement_regex.match(child.text):
                            # print('not a measurement')
                            if child.dep_ in ['compound', 'amod']:
                                # print('found a compound')
                                self.ingredient += child.text + " "

                    # if token.index() == -1:
                    #   self.ingredient += token.text
                    # else:
                      # self.ingredient += token.text + ", " 

                    self.ingredient += token.text + ", "

                if token.pos_ == 'NUM':  # sometimes the amount is registered as nsubj
                    try:
                        self.amount = Fraction(token.text)
                    except ValueError as err:
                        print("ValueError: {}".format(err))
            # TODO: determine the measurement (if any)
            elif token.pos_ == 'NUM':
                print("the amount is {}".format(token))
                try:
                    self.amount = Fraction(token.text)
                except ValueError as err:
                    print("ValueError: {}".format(err))
            elif token.pos_ == 'X' and token.dep_ == 'nummod':
                print("the amount is {}".format(token))
                try:
                    self.amount = Fraction(token.text)
                except ValueError as err:
                    print("ValueError: {}".format(err))



if __name__ == '__main__':
    training_set = []
    training_set_ingredients = []

    recipe_nlp = RecipeNLP()

    for line in df_sample_2['ingredients']:
        parsed_line = recipe_nlp.parse(line)
        training_set.append((line, [parsed_line.amount, parsed_line.measurement, parsed_line.ingredient]))
        training_set_ingredients.append(parsed_line.ingredient)

    pp = pprint.PrettyPrinter()
    pp.pprint(training_set_ingredients)

    with open('annotated_data_old.py', 'w') as annotated_file:
      annotated_file.write(pp.pformat(training_set))

Streaming output truncated to the last 5000 lines.
18 nummod NUM
the amount is 18
teaspoon compound NOUN
thyme appos NOUN
( punct PUNCT
optional appos ADJ
) punct PUNCT
' punct PUNCT
] punct PUNCT

[ ' 12 cup plum , stone and chop ' , ' 1 cup prune , pit and chop ' , ' 1 cup low - fat vanilla yogurt ' , ' 12 cup buttermilk ' , ' 1 14 cup soymilk , calcium enrich ' , ' 8 large ice cube ' ]
[ punct PUNCT
' punct NUM
the amount is '
ValueError: Invalid literal for Fraction: "'"
12 nummod NUM
the amount is 12
cup compound NOUN
plum nmod NOUN
, punct PUNCT
stone conj NOUN
and cc CCONJ
chop conj NOUN
' punct PUNCT
, punct PUNCT
' punct PUNCT
1 nummod NUM
the amount is 1
cup compound NOUN
prune appos NOUN
, punct PUNCT
pit conj NOUN
and cc CCONJ
chop conj NOUN
' punct PUNCT
, punct PUNCT
' punct PUNCT
1 nummod NUM
the amount is 1
cup compound NOUN
low amod ADJ
- punct PUNCT
fat compound NOUN
vanilla compound NOUN
yogurt ROOT PROPN
' punct PUNCT
, punct PUNCT
' punct PUNCT
12 nummod NUM
the am

In [ ]:
df_sample_2['ing'] = training_set_ingredients
df_sample_2.head(50)

<ipython-input-16-092f6880c662>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_2['ing'] = training_set_ingredients


,title,instructions,ingredients,ing
0,dilly macaroni salad recipe,['Cook macaroni according to package direction...,"[ ' 1 c. elbow macaroni ' , ' 1 c. cube americ...","elbow macaroni, american cheese, slice celery,..."
1,gazpacho,['Add the tomatoes to a food processor with a ...,"[ ' 8 tomato , quarter ' , ' kosher salt ' , '...","red onion, green bell pepper, red bell pepper,..."
2,crunchy onion potato bake,"['Preheat oven to 350 degrees Fahrenheit.', 'S...","[ ' 2 12 cup milk ' , ' 1 12 cup water ' , ' 1...","milk, butter, homestyle, whole kernel corn, dr..."
3,cool 'n easy creamy watermelon pie,"['Dissolve Jello in boiling water.', 'Allow to...",[ ' 1 ( 3 ounce ) package watermelon gelatin '...,"package, Cool Whip, thaw, seedless watermelon,..."
4,strawberry rhubarb dump cake,['Put ingredients in a buttered 9 x 12 x 2-inc...,"[ ' 6 -8 cup fresh rhubarb , or ' , ' 6 -8 cup...","frozen rhubarb, thaw, sugar, strawberry O gela..."
5,yogurt parfaits,['Layer all ingredients in a serving dish.'],"[ ' 8 ounce , weight Light Fat Free Vanilla Yo...","weight Light Vanilla Yogurt, Activia, Fresh Sl..."
6,pink sangria,"['In a pitcher, combine all the ingredients ex...","[ ' 1 ( 750 ml ) bottle rise wine , chill ' , ...","bottle, rise wine, brandy, Grand Marnier, cran..."
7,pineapple loaf,"['Cream butter, sugar and egg.', 'Add orange j...","[ ' 14 cup butter ' , ' 34 cup brown sugar ' ,...","butter, brown sugar, egg, juice concentrate, g..."
8,brown sugar 'karintou' snacks,"['In a bowl, mix flour, baking powder, sugar, ...","[ ' 200 gram Cake flour ' , ' 1 tsp bake powde...","gram Cake flour, tsp bake powder, gram Sugar, ..."
9,corn casserole,"['Fry hamburger and onion together.', 'Peel an...","[ ' 1 can tomato sauce ' , ' 1 can corn ' , ' ...","tomato, kg, medium cheddar cheese, chop, hambu..."


In [ ]:
df_sample_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         5000 non-null   object
 1   instructions  5000 non-null   object
 2   ingredients   5000 non-null   object
 3   ing           5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [ ]:
df_sample_2.to_csv('/content/drive/MyDrive/Project 298/Final Project Files/EDA Notebooks/recipe_data_5000.csv', index=False)